# Figure 3: Intensity and Duration in FOSI/Obs

In [ ]:
# general use packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

import regionmask
import geopandas as gp
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# from @GlacialMeg

import matplotlib as mpl
# Font style and size
plt.rcParams['font.family'] = 'Arial'         # Font
plt.rcParams['font.size'] = 10                # General font size unless set below
plt.rcParams['axes.labelsize'] = 11           # Axes labels font size
plt.rcParams['figure.titlesize'] = 12         # Title font size
plt.rcParams['figure.titleweight'] = 'bold'   # Bold title
plt.rcParams['axes.labelweight'] = 'bold'     # Bold axes labels
    
# Axes and ticks parameters
plt.rcParams['axes.linewidth'] = 1            # Width of axes border
plt.rcParams['xtick.direction'] = 'in'        # Make x ticks go in
plt.rcParams['ytick.direction'] = 'in'        # Make y ticks go in
plt.rcParams['xtick.major.size'] = 5          # Set x tick length 
plt.rcParams['ytick.major.size'] = 5          # Set y tick length
plt.rcParams['xtick.major.width'] = 1         # Set x tick width 
plt.rcParams['ytick.major.width'] = 1         # Set y tick width

# Line style
plt.rcParams['lines.linewidth'] = 1           # Set line widths on plots
plt.rcParams['lines.linestyle'] = '-'         # Set line styles on plots

# Math text font characteristics
plt.rcParams['mathtext.fontset'] = 'cm'       # Choose font for math text
plt.rcParams['mathtext.default'] = 'regular'  # Make math text not bold or italic
# mathtext.FontConstantsBase.sup1 = 0.4         # Move superscript text to a better height

# For showing plots on GitHub
%matplotlib inline
plt.rcParams['figure.dpi']= 100

In [ ]:
def detrend_linear(dat, dim):
    """ linear detrend dat along the axis dim """
    params = dat.polyfit(dim=dim, deg=1)
    fit = xr.polyval(dat[dim], params.polyfit_coefficients)
    dat = dat-fit
    return dat

## Obs and FOSI

### process here (no rolling)

In [ ]:
obs = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')

In [ ]:
fosi_omega = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/omega_arag.monthly.surface.regrid.nc')['omega_arag']
fosi_temp = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/TEMP.monthly.surface.regrid.nc')['TEMP']
fosi_h = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/pH_3D.monthly.surface.regrid.nc')['pH_3D']; fosi_h = 10**(-fosi_h)

In [ ]:
fosi_omega['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
fosi_temp['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
fosi_h['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
obs_omega = obs.omega_ar
obs_temp = obs.temperature
obs_h = 10**(-obs.ph_total)

In [ ]:
obs_omega, fosi_omega = xr.align(obs_omega, fosi_omega)
obs_temp, fosi_temp = xr.align(obs_temp, fosi_temp)
obs_h, fosi_h = xr.align(obs_h, fosi_h)

In [ ]:
mask = np.isnan(obs_omega.isel(time=0))#obs_omega.isel(time=0).where(obs_omega.isel(time=0)==np.NaN,obs_omega.isel(time=0))
mask = mask.drop('time')

In [ ]:
fosi_omega = fosi_omega.where(mask == 0, np.NaN)
fosi_temp = fosi_temp.where(mask == 0, np.NaN)
fosi_h = fosi_h.where(mask == 0, np.NaN)

### remove climatology and trend

In [ ]:
def detrend_linear(dat, dim):
    """ linear detrend dat along the axis dim """
    params = dat.polyfit(dim=dim, deg=1)
    fit = xr.polyval(dat[dim], params.polyfit_coefficients)
    dat = dat-fit
    return dat

In [ ]:
# climatology
fosi_omega = fosi_omega.groupby('time.month') - fosi_omega.groupby('time.month').mean()
fosi_temp = fosi_temp.groupby('time.month') - fosi_temp.groupby('time.month').mean()
fosi_h = fosi_h.groupby('time.month') - fosi_h.groupby('time.month').mean()

obs_omega = obs_omega.groupby('time.month') - obs_omega.groupby('time.month').mean()
obs_temp = obs_temp.groupby('time.month') - obs_temp.groupby('time.month').mean()
obs_h = obs_h.groupby('time.month') - obs_h.groupby('time.month').mean()

In [ ]:
# trend
fosi_omega = detrend_linear(fosi_omega,'time')
fosi_temp = detrend_linear(fosi_temp,'time')
fosi_h = detrend_linear(fosi_h,'time')

obs_omega = detrend_linear(obs_omega,'time')
obs_temp = detrend_linear(obs_temp,'time')
obs_h = detrend_linear(obs_h,'time')

## define extremes (no rolling)

In [ ]:
# level = 0.1
# obs_omega_thold = obs_omega.quantile(level)
# fosi_omega_thold = fosi_omega.quantile(level)

# obs_omega_extreme = obs_omega.where(obs_omega < obs_omega_thold)
# fosi_omega_extreme = obs_omega.where(fosi_omega < fosi_omega_thold)

# obs_omega_extreme = ~np.isnan(obs_omega_extreme)
# fosi_omega_extreme = ~np.isnan(fosi_omega_extreme)

# level = 0.9
# obs_temp_thold = obs_temp.quantile(level)
# fosi_temp_thold = fosi_temp.quantile(level)

# obs_temp_extreme = obs_temp.where(obs_temp > obs_temp_thold)
# fosi_temp_extreme = obs_temp.where(fosi_temp > fosi_temp_thold)

# obs_temp_extreme = ~np.isnan(obs_temp_extreme)
# fosi_temp_extreme = ~np.isnan(fosi_temp_extreme)

# level = 0.9
# obs_h_thold = obs_h.quantile(level)
# fosi_h_thold = fosi_h.quantile(level)

# obs_h_extreme = obs_h.where(obs_h > obs_h_thold)
# fosi_h_extreme = obs_h.where(fosi_h > fosi_h_thold)

# obs_h_extreme = ~np.isnan(obs_h_extreme)
# fosi_h_extreme = ~np.isnan(fosi_h_extreme)

In [ ]:
# rolling 
var='omega_arag'
obs_extremes = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.obs.rolling.thold.Rolling.nc')['threshold']

## Load SMYLE

In [ ]:
# obs_pac = obs_omega_extreme.sel(lat=0.5,lon=-130.5).astype(int)

In [ ]:
var = 'omega_arag'
var2 = 'CO3'
depth = 'surface'
# init = '05'

smyle02 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.02.binary.Rolling.full.2.nc')['binary']
smyle02_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.02.time.nc')#.sel(L=slice(1,12))

smyle05 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.05.binary.Rolling.full.2.nc')['binary']
smyle05_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.05.time.nc')#.sel(L=slice(1,12))

smyle08 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.08.binary.Rolling.full.2.nc')['binary']
smyle08_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')#.sel(L=slice(1,12))

smyle11 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.11.binary.Rolling.full.2.nc')['binary']
smyle11_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.11.time.nc')#.sel(L=slice(1,12))

smyle02 = smyle02.expand_dims("init"); smyle02_time  = smyle02_time.expand_dims("init")
smyle05 = smyle05.expand_dims("init"); smyle05_time  = smyle05_time.expand_dims("init")
smyle08 = smyle08.expand_dims("init"); smyle08_time  = smyle08_time.expand_dims("init")
smyle11 = smyle11.expand_dims("init"); smyle11_time  = smyle11_time.expand_dims("init")

smyle02['init'] = ['02']; smyle02_time['init'] = ['02']
smyle05['init'] = ['05']; smyle05_time['init'] = ['05']
smyle08['init'] = ['08']; smyle08_time['init'] = ['08']
smyle11['init'] = ['11']; smyle11_time['init'] = ['11']

smyle = xr.concat([smyle02,smyle05,smyle08,smyle11],'init')
smyle_time = xr.concat([smyle02_time,smyle05_time,smyle08_time,smyle11_time],'init')

In [ ]:
# smyle_avg = smyle.mean('M')

In [ ]:
obs_extremes.time.dt.year[-1]

In [ ]:
smyle = smyle.sel(Y = slice(obs_extremes.time.dt.year[0], obs_extremes.time.dt.year[-1]))

In [ ]:
smyle.isel(L=0,M=0,Y=0,init=0).plot()

In [ ]:
plt.plot(smyle.sel(Y=1995,lat=0.5,lon=-120.5).sel(init='02').isel(M=1).values)

plt.plot(obs_extremes.sel(time=slice('1995-02','1997-01'),lat=0.5,lon=-120.5))

## mask

In [ ]:
mask = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')['temperature'].isel(time=0).drop('time')
mask = mask.where(mask.lat < 65)

In [ ]:
obs_extremes = obs_extremes.where(np.isnan(mask) == 0, np.NaN)

In [ ]:
smyle = smyle.where(np.isnan(mask) == 0, np.NaN)

## For duration

In [ ]:
smyle_use = smyle#.isel(M=1,init=0)

In [ ]:
%%time
#what dimension do I do the diff over?
event_boundaries_smyle = smyle_use.astype(int).diff(dim='L')

distinct_event_count_smyle = (event_boundaries_smyle == -1).sum(dim=('L','Y','M','init'))
# distinct_event_count_smyle = (event_boundaries_smyle == -1).sum(dim=('L','Y','init'))
# distinct_event_count_smyle = (event_boundaries_smyle == -1).sum(dim=('L','Y'))

avg_length_smyle = smyle.astype(int).sum(('L','M','Y','init')) / distinct_event_count_smyle#.where(distinct_event_count_smyle > 0, np.NaN)
# avg_length_smyle = smyle_use.astype(int).sum(('L','Y','init')) / distinct_event_count_smyle#.where(distinct_event_count_smyle > 0, np.NaN)
# avg_length_smyle = smyle_use.astype(int).sum(('L','Y')) / distinct_event_count_smyle#.where(distinct_event_count_smyle > 0, np.NaN)



In [ ]:
(smyle.astype(int).sum(('L','M','Y','init')) / distinct_event_count_smyle).plot()

In [ ]:
distinct_event_count_smyle.plot()

In [ ]:
avg_length_smyle.plot(vmax=7)

In [ ]:
distinct_event_count_smyle.plot()

In [ ]:
length_smyle = len(event_boundaries_smyle.L) * len(event_boundaries_smyle.Y) * len(event_boundaries_smyle.M) * len(event_boundaries_smyle.init)
# length_smyle = len(event_boundaries_smyle.L) + len(event_boundaries_smyle.Y) + len(event_boundaries_smyle.M) + len(event_boundaries_smyle.init)
# length_smyle = len(event_boundaries_smyle.L) + len(event_boundaries_smyle.Y) + len(event_boundaries_smyle.init)

In [ ]:
length_smyle

In [ ]:
above_zero = obs_extremes > 0

# Find when the mask transitions from True to False or vice versa
event_boundaries = above_zero.astype(int).diff(dim='time')

# Count the number of distinct events
distinct_event_count = (event_boundaries == -1).sum(dim='time')
distinct_event_count

avg_length = above_zero.sum('time') / (distinct_event_count)#.where(distinct_event_count > 0, np.NaN)

In [ ]:
(avg_length).plot()

In [ ]:
# Find the slope and intercept of the best fit line
slope, intercept = np.polyfit(np.arange(0,100), np.arange(0,100), 1)

# Create a list of values in the best fit line
abline_values = [slope * i + intercept for i in np.arange(0,100)]

In [ ]:
avg_length.plot()
plt.show()
# avg_length_smyle.plot()
# plt.show()

In [ ]:
f,ax = plt.subplots(1,1)
plt.grid(zorder=0)
plt.scatter(avg_length_smyle.stack(lat_lon=('lat', 'lon')), avg_length.stack(lat_lon=('lat', 'lon')),color='grey',s=5,zorder=4)
plt.plot(abline_values,abline_values,'--',color='black',linewidth=1,zorder=1)
plt.xlim(0,10)
plt.ylim(0,10)
plt.xlabel('predicted duration (months)')
plt.ylabel('observed duration (months)')

corr = xr.corr(avg_length, avg_length_smyle)
plt.title(var + ': r = ' + str(np.round(corr.values,2)))

# # f.savefig('./figures/' + var + '.duration.pdf')

In [ ]:
f,ax = plt.subplots(1,1)
plt.grid(zorder=0)
plt.scatter(distinct_event_count_smyle.stack(lat_lon=('lat', 'lon'))/length_smyle, distinct_event_count.stack(lat_lon=('lat', 'lon'))/468,color='grey',s=5,zorder=4)
plt.plot(abline_values,abline_values,'--',color='black',linewidth=1,zorder=1)
plt.xlim(0,0.16)
plt.ylim(0,0.16)
plt.xlabel('predicted number of extremes')
plt.ylabel('observed number of extremes')

corr = xr.corr(distinct_event_count, distinct_event_count_smyle)
plt.title(var + ': r = ' + str(np.round(corr.values,2)))

f.savefig('./figures/' + var + '.number.pdf')

## Intensity

In [ ]:
var = 'ph_total'
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')[var]
ds = 10**(-ds)

ds = ds.groupby('time.month') - ds.groupby('time.month').mean()

ds_detr = detrend_linear(ds,dim='time')

In [ ]:
ds_detr = ds_detr.where(np.isnan(mask) == 0, np.NaN)

In [ ]:
# %%time
# # SMYLE Utility functions
# from SMYLEutils import io_utils as io
# from SMYLEutils import calendar_utils as cal
# from SMYLEutils import stat_utils as stat
# # # create and save SMYLE combined for all inits

# var = 'pH_3D'
# var2 = 'pH_3D' # var
# depth = 'surface'

# smyle02 = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var+  '.monthly.' + depth + '.02.regrid.nc')[var]
# smyle02_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.02.time.nc')
# smyle02 = smyle02.drop('time')
# smyle02 = smyle02.sel(L=slice(1,12))

# smyle05 = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var+  '.monthly.' + depth + '.05.regrid.nc')[var]
# smyle05_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.05.time.nc')
# smyle05 = smyle05.drop('time')
# smyle05 = smyle05.sel(L=slice(1,12))

# smyle08 = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.08.regrid.nc')[var]
# smyle08_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')
# smyle08 = smyle08.drop('time')
# smyle08 = smyle08.sel(L=slice(1,12))

# smyle11 = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.11.regrid.nc')[var]
# smyle11_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.11.time.nc')
# smyle11 = smyle11.drop('time')
# smyle11 = smyle11.sel(L=slice(1,12))

# smyle02 = 10**(-smyle02)
# smyle05 = 10**(-smyle05)
# smyle08 = 10**(-smyle08)
# smyle11 = 10**(-smyle11)

# print('deseasoning')
# # %%time
# smyle02_anom,smyle02_clim = stat.remove_drift(smyle02,smyle02_time,1982,2023)
# smyle05_anom,smyle05_clim = stat.remove_drift(smyle05,smyle05_time,1982,2023)
# smyle08_anom,smyle08_clim = stat.remove_drift(smyle08,smyle08_time,1982,2023)
# smyle11_anom,smyle11_clim = stat.remove_drift(smyle11,smyle11_time,1982,2023)

# print('detrending')
# # %%time
# smyle02_anom = detrend_linear(smyle02_anom.time,'Y')
# smyle05_anom = detrend_linear(smyle05_anom.time,'Y')
# smyle08_anom = detrend_linear(smyle08_anom.time,'Y')
# smyle11_anom = detrend_linear(smyle11_anom.time,'Y')

# # del smyle, smyle_time
# # del smyle02_anom, smyle05_anom, smyle08_anom, smyle11_anom, smyle02, smyle05, smyle08, smyle11

# print('combining')
# # %%time
# smyle02_anom = smyle02_anom.expand_dims("init"); smyle02_time  = smyle02_time.expand_dims("init")
# smyle05_anom = smyle05_anom.expand_dims("init"); smyle05_time  = smyle05_time.expand_dims("init")
# smyle08_anom = smyle08_anom.expand_dims("init"); smyle08_time  = smyle08_time.expand_dims("init")
# smyle11_anom = smyle11_anom.expand_dims("init"); smyle11_time  = smyle11_time.expand_dims("init")

# smyle = xr.concat([smyle02_anom,smyle05_anom,smyle08_anom,smyle11_anom],'init')
# smyle_time = xr.concat([smyle02_time,smyle05_time,smyle08_time,smyle11_time],'init')

# smyle['init'] = smyle['init']

# # print('converting')
# # smyle = 10**(-smyle)

# var = 'H+'
# smyle.to_netcdf('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.all_init.regrid3.nc')
# smyle_time.to_netcdf('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.all_init.time3.nc')

# # smyle.isel(M=0,L=0,init=0,Y=20).plot()

In [ ]:
# open smyle data
var = 'H+'
smyle_ds = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.all_init.regrid3.nc')['__xarray_dataarray_variable__']
smyle_ds_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var +  '.monthly.' + depth + '.all_init.time3.nc')['time']


In [ ]:
smyle_ds['init'] = smyle['init'].values
smyle_ds_time['init'] = smyle_time['init'].values

In [ ]:
smyle_ds = smyle_ds.where(np.isnan(mask) == 0, np.NaN)

In [ ]:
smyle_ds = smyle_ds.sel(Y = slice(obs_extremes.time.dt.year[0], obs_extremes.time.dt.year[-1]))

In [ ]:
smyle_ds.isel(init=0,Y=30,M=0,L=0).plot()

In [ ]:
smyle.isel(init=0,Y=30,M=0,L=0).plot()

In [ ]:
smyle_ds_ex = smyle_ds.where(smyle > 0)

In [ ]:
smyle_ds_ex.sel(Y=1990).isel(L=0,M=0,init=1).plot()

In [ ]:
obs_ds_ex = ds_detr.where(obs_extremes > 0)

In [ ]:
obs_ds_ex.isel(time=101).plot()

In [ ]:
weights = np.cos(np.deg2rad(obs_ds_ex.lat))

In [ ]:
obs_ds_ex_avg = obs_ds_ex.weighted(weights).mean(('lat','lon'))

In [ ]:
smyle_ds_ex_avg = smyle_ds_ex.weighted(weights).mean(('lat','lon'))

In [ ]:
obs_ds_ex_time = obs_ds_ex.mean('time')

In [ ]:
obs_ds_ex.max('time').plot()

In [ ]:
smyle_ds_ex_time = smyle_ds_ex.mean(("L",'M',"Y","init"))

In [ ]:
smyle_ds_ex.isel(init=0,Y=10,M=10,L=1).plot()

In [ ]:
# Find the slope and intercept of the best fit line
slope, intercept = np.polyfit(np.arange(0,100), np.arange(0,100), 1)

# Create a list of values in the best fit line
abline_values = [slope * i + intercept for i in np.arange(0,100)]

In [ ]:
smyle_ds_ex_time.sel(lat=0.5,lon=-130.5)

In [ ]:
f,ax = plt.subplots(1,1)
plt.grid(zorder=0)
plt.scatter(abs(smyle_ds_ex_time.stack(lat_lon=('lat', 'lon'))), abs(obs_ds_ex_time.stack(lat_lon=('lat', 'lon'))),color='grey',s=5,zorder=4)
plt.plot(abline_values,abline_values,'--',color='black',linewidth=1,zorder=1)
plt.xlim(0,0.5e-9)
plt.ylim(0,0.5e-9)
plt.xlabel('predicted intensity')
plt.ylabel('observed intensity')

corr = xr.corr(smyle_ds_ex_time, obs_ds_ex_time)
plt.title(var + ': r = ' + str(np.round(corr.values,2)))

f.savefig('./figures/' + var + '.intensity.pdf')

In [ ]:
obs_ds_ex_time.plot()
plt.show()

smyle_ds_ex_time.plot()
plt.show()

## Do Forecasts predict OAX before the initial appearance of one?

In [ ]:
# use SMYLE and observed extremes


## old stuff

### avg. length method 1 - different, going with method 2 (easier)

In [ ]:
# test case 
tmp = event_boundaries.sel(lat=10.5,lon=-150.5).where(event_boundaries.sel(lat=10.5,lon=-150.5) != 0).dropna('time')

time_1 = tmp.where(tmp == 1).dropna('time')
time_neg1 = tmp.where(tmp == -1).dropna('time')

count = -(time_1.time.values - time_neg1.time.values).astype('timedelta64[M]').astype(int)

count.mean()
count.sum() / len(time_neg1)

count

if count[0] < 0:
    avg_length = (len(event_boundaries.time) + count.sum()) / len(time_neg1)
else: 
    avg_length = count.sum() / len(time_neg1)

In [ ]:
len(time_neg1)

In [ ]:
count.sum() / len(count)

In [ ]:
len(time_1)

In [ ]:
print(avg_length)

In [ ]:
# compare
tst1 = count
tst2 = (above_zero.sel(lat=10.5,lon=-150.5).where(above_zero.sel(lat=10.5,lon=-150.5) > 0, drop=True))


In [ ]:
count.sum()

In [ ]:
tst2.sum()

In [ ]:
# len(event_boundaries.time) + count.sum()

In [ ]:
tst2.sum()

### avg length method 2 - use this!

In [ ]:
above_zero.sel(lat=0.5,lon=-150.5).where(above_zero.sel(lat=0.5,lon=-150.5) > 0, drop=True)

In [ ]:
above_zero.sel(lat=10.5,lon=-150.5).where(above_zero.sel(lat=10.5,lon=-150.5) > 0, drop=True).sum()

In [ ]:
len(above_zero.sel(lat=10.5,lon=-150.5).where(above_zero.sel(lat=10.5,lon=-150.5) > 0, drop=True))

In [ ]:
len(above_zero.sel(lat=10.5,lon=-150.5).where(above_zero.sel(lat=10.5,lon=-150.5) > 0, drop=True)) / distinct_event_count.sel(lat=10.5,lon=-150.5)


In [ ]:
(above_zero.sel(lat=10.5,lon=-150.5).where(above_zero.sel(lat=10.5,lon=-150.5) > 0, drop=True)).sum() / distinct_event_count.sel(lat=10.5,lon=-150.5)


In [ ]:
avg_length2 = (above_zero.sum('time') / distinct_event_count).sel(lat=0.5,lon=-150.5)